### CRUD Integration Checkpoint (01/27/2025)

**Executive Summary:**
- Made a database to store user chess games and the moves played in the form of PGN files.
- Created a REST api with POST, GET, PATCH, and DELETE requests to manipulate data.
- Created manager page on frontend to review, delete, and update games.
- Created function request to save games automatically after completion.

**Team Purpose:**
- Chess social platform to discuss, analyze and learn chess content. Our project will be a social media website and forum where users can discuss chess topics and interact with Artificial Intelligence to learn and play chess.

**Individual Features:**
- The purpose of my feature for this review is to store games the user plays against the computer, this enables future review and analysis using the analysis page. The manager page employs CRUD functions so the user can manage the data.
- The game feature enables the user to play chess and provide data and topics of discussion for the users.

**Input/Output requests:**
- The PGN API has GET, POST, PATCH, and DELETE requests. The GET request is seen in the manager when the page is loaded as all the games are populated in cards, the POST request is made on the game page immediately after the game is over. The PATCH request is used to update the database entries, a PATCH request is used instead of a PUT as a PUT typically means all fields are changed, but for my data that would be illogical do I developed a PATCH method instead. Lastly, the DELETE request can also be used from the manager page. (to be shown in the live demo)

```
class _BULK_CRUD(Resource):
        def get(self):
            pgns = Pgn.query.all()
            json_ready = []
            for pgn in pgns:
                pgn_data = pgn.read()
                json_ready.append(pgn_data)
            return jsonify(json_ready)
```

```
class _CRUD(Resource):
        def post(self):
            body = request.get_json()
            body_pgn = body['pgn']
            body_name = body['name']
            body_id = body['user_id']

            ### TODO: Add error handling error for various input fields

            pgn_obj = Pgn(pgn=body['pgn'], date=body['date'], name=body['name'], user_id=body['user_id'])
            pgn_obj.create()
            # if not pgn:  # failure returns error message
            #     return {'message': f'Processed {body_pgn}, either a format error or User ID {body_id} is duplicate'}, 400

            return jsonify(pgn_obj.read())

        def delete(self):
            body = request.get_json()
            pgn = Pgn.query.get(body['id'])
            if not pgn:
                return {'message': 'Pgn not found'}, 404
            pgn.delete()
            return jsonify({"message": "pgn deleted"})

        def patch(self):
            body = request.get_json()
            pgn = Pgn.query.get(body['id'])
            if pgn is None:
                return {'message': 'Pgn not found'}, 404

            if 'name' in body:
                pgn._name = body['name']

            pgn.patch()
            return jsonify(pgn.read())
```

- Postman requests work as well, and if breakpoint is placed on the return line the function will cease to function, displaying proper setup.

**Requests & DOM:**
- The fetch request receives data from the REST API and is manipulated in the DOM (Document Object Model). Looping through the response object, and then accessing each property of each individual “game” to then be displayed.

```
async function fetchGames(){
        try {
            const response = await fetch(`${pythonURI}/api/pgns`, fetchOptions);
            if (!response.ok) {
                throw new Error('Failed to fetch groups: ' + response.statusText);
            }

            const games = await response.json();
            const container = document.getElementById('cards-container');
            container.innerHTML = '';

            games.forEach(game => {
                const card = document.createElement('div');
                card.className = 'card';
                card.innerHTML = `
                    <h3>${game.name}</h3>
                    <p>${game.pgn}</p>
                    <button class="button update">Update</button>
                    <button class="button delete">Delete</button>
                    <button class="button analyze">Analyze</button>
                `;

                card.querySelector('.delete').addEventListener('click', () => deleteGame(game.id));
                card.querySelector('.update').addEventListener('click', () => updateGame(game.id));
                card.querySelector('.analyze').addEventListener('click', () => redirectToAnalyze(game.id));
                container.appendChild(card);
            });
        } catch (error) {
            console.error('Error fetching entries:', error);
        }
    }
```

- The queries to database are made by Flask_SQLAlchemy which is an ORM (Object Relational Manager) that abstracts the SQL requests enabling easier interfacing with the database.

```
    def delete(self):
        try:
            db.session.delete(self)
            db.session.commit()
        except Exception as e:
            db.session.rollback()
            raise e
    
    def patch(self):
        inputs = Pgn.query.get(self.id)

        if inputs._pgn:
            self._pgn = inputs._pgn
        if inputs._date:
            self._date = inputs._date
        if inputs._name:
            self._name = inputs._name

        try:
            db.session.commit()
        except IntegrityError:
            db.session.rollback()
            logging.warning(f"IntegrityError: Could not update pgn entry.")
            return None
        return self
```

- The classes in my API file have multiple methods that can be called, most process the request data, parse it, perform the associated database operation and return a response. The database operations that create, delete, read, etc, are defined in the model file.

**Algorithmic Request:**
- My “_CRUD” class handles POST, DELETE, and PATCH Requests. Based on the fetch request parameters one of the methods is called the corresponding logic executed. The function take the request data, parse it if necessary call the database function required and return a response (status like 200, or 404, or something else.)
- The delete method, obtains the entry id to be deleted from the fetch request, then it searches through the database to find the entry and delete it. The development of the function involved iteration, as I tested what parameters to be sent from frontend, in the end the entry id was most logical and efficient.
- The body of the request contains the id, and the response contains a status message and status code. Should the call be successful, it will say “PGN deleted” with a 200 status code, else it returns “PGN could not be found” with a 404 error code.

**Call to Algorithm Request:**
- The delete request is called when a click is made on the delete button, an event listener listens for a click action and calls the delete function accordingly which makes the fetch request. The fetch makes a DELETE call to /api/pgn and passes a body with an id to be deleted. The fetch request is a try-except block to handle errors.
- After the response is returned, should there be no issues the function that runs the GET request will be called to update the page after the delete request, else error messages are printed.
- The sole parameter is the entry’s id, and the jsonify response is a a string indicating success or error along with the corresponding HTTP status code.

```
 async function updateGame(id) {
        const newName = prompt("Enter the new name for the game:");
        try {
            const response = await fetch(`${pythonURI}/api/pgn`, {
                method: 'PATCH',
                headers: {
                    'Accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ id: id, name: newName })
            });
            if (!response.ok) {
                throw new Error('Failed to delete: ' + response.statusText);
            }

            fetchGames();
        } catch (error) {
            console.error('Error deleting entry:', error);
        }
    }
```



